In [1]:
import os
import operator
import random

import pandas as pd
import numpy as np
import tensorflow.contrib.keras as keras
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

from vixstructure.utils import parse_model_repr
from vixstructure.data import LongPricesDataset

In [4]:
directory, _, files = next(os.walk("models/experiment05"))
h5_files = tuple(file for file in files if os.path.splitext(file)[1] == ".h5")
csv_files = tuple(file for file in files if os.path.splitext(file)[1] == ".csv")
stats = [(parse_model_repr(os.path.splitext(file)[0]),
          pd.read_csv(os.path.join(directory, file), header=0, index_col=0,
                      dtype={"epoch":int, "loss":np.float32, "val_loss":np.float32,
                             "denorm_mse":np.float32, "val_denorm_mse":np.float32}))
         for file in csv_files]
parameters, stats_data = zip(*stats)
for d in stats_data:
    if len(d.columns) == 2:
        use_this_as_columns = d.columns
        break
for d in stats_data:
    if len(d.columns) == 4:
        d.drop(["loss", "val_loss"], axis=1, inplace=True)
        d.columns = use_this_as_columns
dataframe = pd.concat(stats_data, keys=[(p.optimizer, p.depth, p.width, p.normalized) for p in parameters],
                      names=["optimizer", "depth", "width", "normalized"])

Index(['loss', 'val_loss'], dtype='object')


In [9]:
dataframe.loc["SGD"].min()

loss        0.142136
val_loss    0.116014
dtype: float32

In [10]:
dataframe.loc["Adam"].min()

loss        0.034315
val_loss    0.081978
dtype: float32